# Game Plan #

### For the perfect game at the perfect price ###


#### Colin Macy

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Implementation](#3.-Implementation)
1. [Results and Discussion](#4.-Results-and-discussion)
1. [Conclusion](#5.-Conclusion)
1. [Author Contributions](#6-author-contributions)

## 1. Introduction ##

Pathfinding is a critical concept in robotics, game development, and autonomous navigation. This project first uses the A* algorithm to find the shortest path on a map and then applies QP (Quadratic Programming) to smooth the route. In game maps, a smooth path can make character movement more natural and visually appealing. For example, in the game Baldur’s Gate, players click on a destination, and the character autonomously chooses the optimal path to reach the clicked location. Sharp turns can make character movement appear rigid. This project will use maps from Baldur’s Gate II for experimentation.

The A* algorithm is a highly popular pathfinding algorithm that helps find the shortest distance between two objects. The algorithm uses a heuristic that measures the cost from the current position to the destination position. 

The A* (A-star) algorithm is a widely used pathfinding and graph traversal algorithm. It is designed to find the shortest path between a start node and a goal node in a weighted graph. A* combines the strengths of both Dijkstra's algorithm (which guarantees the shortest path by exploring all nodes) and greedy best-first search (which makes decisions based on heuristic estimates of the goal). This hybrid approach allows A* to efficiently find the optimal path while minimizing the search space, making it particularly suitable for real-time applications like video games. The above plot demonstrates the purpose for A* and serves as a basis for what we hope to implement within a game-environment.

For a deeper understanding of how the A* algorithm works, you can refer to the work by Yumeng Yan, who discusses its practical applications in his paper.
Yan, Y. (2024). Research on the A Star Algorithm for Finding Shortest Path. Beijing-Dublin International College at BJUT, Beijing University of Technology. Retrieved from [[Research Gate](https://www.researchgate.net/publication/370573811_Research_on_the_A_Star_Algorithm_for_Finding_Shortest_Path)]

<div style="display: flex; justify-content: center; gap: 20px; text-align: center;">
    <div>
        <img src="./AR0011SR.png" style="transform: scaleY(-1);" alt="MAP1" title="MAP1" width="250"/>
        <p><em>Baldur's Gate Map</em></p>
    </div>
    <div>
        <img src="./path.png" alt="MAP2" title="MAP2" width="300"/>
        <p><em>Baldur's Gate Pathing</em></p>
    </div>
    <div>
        <img src="./zoomed.png" alt="MAP3" title="MAP3" width="300"/>
        <p><em>Baldur's Gate Zoomed-In Pathing</em></p>
    </div>
</div>
The images above illustrate the ultimate goal of our model. In the first image, we visualize obstacles (depicted in black), set within a game map, that our model must navigate through smoothly and efficiently. Using the A* algorithm, our system will identify the shortest path between the start and goal points, which we then further optimize using a Quadratic Programming (QP) approach to smooth the route. This is demonstrated in the second and third images.

In the second image, you can clearly observe the path that the A* algorithm has determined across the map. The third image provides a closer look at how our smoothing optimizer enhances this path by focusing on a zoomed-in section, where the smoothed path is represented in blue. This smoothing process is crucial in making the movement more natural within the game environment. Unlike the rigid, four-way movements often seen in games, our approach offers flexibility, enabling characters to move fluidly in all directions, enhancing realism and gameplay experience.

The map data used for this project was sourced from [Moving AI](https://movingai.com/benchmarks/grids.html), which provides downloadable game maps for testing and development. While our primary focus is on Baldur's Gate, there are additional game maps available, offering opportunities for future adaptations of the model to other games as well.

## 2. Mathematical Model

### Optimization Goal
Our goal is to optimize each point along a path such that the path becomes:
1. **As smooth as possible**: Reduce sharp turns to minimize curvature along the path.
2. **Close to the original path**: Keep the optimized path close to the initial path, ensuring that it maintains the general shape.

#### Smoothness Loss
The smoothness loss is designed to minimize the curvature of the path by using the second-order difference of the path points. 

For a continuous function $f(x)$, its second derivative at a point $x$, denoted $f''(x)$, can be approximated using Taylor expansion. Suppose we have three discrete points $x_{i-1}$, $x_i$, $x_{i+1}$, each separated by a distance $h$. We perform Taylor expansions for $f(x_{i+1})$ and $f(x_{i-1})$:

**Taylor Expansion of $f(x_{i+1})$**

$$
f(x_{i+1}) = f(x_i) + h f'(x_i) + \frac{h^2}{2} f''(x_i) + \frac{h^3}{6} f'''(x_i) + \cdots
$$

**Taylor Expansion of $f(x_{i-1})$**

$$
f(x_{i-1}) = f(x_i) - h f'(x_i) + \frac{h^2}{2} f''(x_i) - \frac{h^3}{6} f'''(x_i) + \cdots
$$

Adding these two expansions cancels the first derivative term $f'(x_i)$:

$$
f(x_{i+1}) + f(x_{i-1}) = 2f(x_i) + h^2 f''(x_i) + \text{higher order terms}
$$

Rearranging terms, we approximate $f''(x_i)$:

$$
f''(x_i) \approx \frac{f(x_{i+1}) - 2f(x_i) + f(x_{i-1})}{h^2}
$$

Thus, when $h = 1$,the second derivative can be approximated by the discrete points $x_{i-1}$, $x_i$, $x_{i+1}$:

$$
f(x_{i+1}) - 2f(x_i) + f(x_{i-1})
$$

In the path smoothing problem, we aim to minimize the curvature at each path point. 

Let us define some variables now:  
- $N$: Total number of path points.  
- $(x[i], y[i])$: The coordinates of the $i$-th path point, where $i \in \{1, \cdots, N\}$.  

The smoothness loss is intended to reduce the curvature of the path. For the $x$-coordinates and $y$-coordinates, the second-order difference is computed as:  

$$
\Delta^2 x[i] = x[i-1] - 2x[i] + x[i+1]
$$

$$
\Delta^2 y[i] = y[i-1] - 2y[i] + y[i+1]
$$

The total smoothness loss for all path points (except the first and last) is defined as:  

$$
\text{smoothness\_loss} = \sum_{i=2}^{N-1} \left( \Delta^2 x[i]^2 + \Delta^2 y[i]^2 \right)
$$

Expanding $\Delta^2 x[i]$ and $\Delta^2 y[i]$, we have:  

$$
\text{smoothness\_loss} = \sum_{i=2}^{N-1} \left( (x[i-1] - 2x[i] + x[i+1])^2 + (y[i-1] - 2y[i] + y[i+1])^2 \right)
$$


#### Proximity Loss
The proximity loss ensures that the optimized path remains close to the original path. Let the coordinates of the original path be $(x_0[i], y_0[i])$, and the optimized path points be $(x[i], y[i])$. The proximity loss is defined as the sum of squared Euclidean distances between each optimized point and its corresponding original point:

$$
\text{proximity\_loss} = \sum_{i=1}^N \left((x[i] - x_0[i])^2 + (y[i] - y_0[i])^2\right)
$$

#### constraints
To ensure that the path begins and ends at the specified locations, we introduce fixed constraints for the start and end points. These constraints anchor the optimized path to the original path's starting and ending points, ensuring that any adjustments made through optimization do not alter the intended entry and exit positions. 

For a given path with coordinates $(x_0[1], y_0[1])$ as the start point and $(x_0[N], y_0[N])$ as the end point, the fixed start and end constraints are defined as follows:

$$
x[1] = x_0[1], \quad y[1] = y_0[1]
$$

$$
x[N] = x_0[N], \quad y[N] = y_0[N]
$$

Obstacle Avoidance (if applicable): Additional constraints can be included to ensure that path points are within feasible (obstacle-free) regions.


### Model Summary


#### Objective Function
We aim to minimize the weighted total loss:

$$
\min \quad \text{smoothness\_loss} + \lambda \times \text{proximity\_loss}
$$

$$
\lambda \in \mathbb{R}_{\geq 0} \text{ is a scalar parameter that controls the trade-off between smoothness and proximity.}
$$

- For $\lambda \approx 0$, the objective places a stronger emphasis on smoothness.
- For larger values of $\lambda$, the objective prioritizes proximity.

#### Decision Variables
The decision variable $(x[i], y[i])$ represents the coordinates of the $i$-th path point, where $i \in \{1, \cdots, N\}$.  
#### Constraints
$$
x[1] = x_0[1], \quad y[1] = y_0[1]
$$

$$
x[N] = x_0[N], \quad y[N] = y_0[N]
$$


### Conversion to Standard QCQP Form
To simplify solving this optimization problem, we convert it to a standard Quadratically Constrained Quadratic Programming (QCQP) form.

#### Variable Definitions
- $N$: Total number of path points.  
- $(x[i], y[i])$: The coordinates of the $i$-th path point, where $i \in \{1, \cdots, N\}$.  
- Decision variable $z \in \mathbb{R}^{2N}$: The stacked decision variable that represents the $x$- and $y$-coordinates of all path points.  

$$
z = [x[1], y[1], x[2], y[2], \ldots, x[N], y[N]]^T \in \mathbb{R}^{2N}
$$

- Given parameter $z_{\text{ref}} \in \mathbb{R}^{2N}$: The reference path, which represents the initial path before optimization.  
- The reference vector $z_{\text{ref}}$ is given as:

$$
z_{\text{ref}} = [x_0[1], y_0[1], x_0[2], y_0[2], \ldots, x_0[N], y_0[N]]^T \in \mathbb{R}^{2N}
$$


#### Objective Function

##### Smoothness Loss
To measure curvature, we calculate the second-order difference for $x$-coordinates and $y$-coordinates.  

The second-order difference for $x[i]$ and $y[i]$ is:  

$$
\text{smoothness\_loss} = \sum_{i=2}^{N-1} \left( (x[i-1] - 2x[i] + x[i+1])^2 + (y[i-1] - 2y[i] + y[i+1])^2 \right)
$$

To represent the second-order difference operation, we can use a discrete second-order difference matrix $D$. For the $x$-coordinates, the matrix $D$ in $\mathbb{R}^{(N-2) \times N}$ is constructed such that each row corresponds to a second-order difference. For example, the $(i-1)$-th row corresponds to the difference: 
$
x[i-1] - 2x[i] + x[i+1]
$

The same structure holds for the $y$-coordinates. By combining the two coordinates, we construct a block-diagonal difference matrix:

$$
\tilde{D} = 
\begin{bmatrix}
D & 0 \\
0 & D 
\end{bmatrix} 
\in \mathbb{R}^{2(N-2) \times 2N}
$$

The smoothness loss can then be written as: 
$$
\text{smoothness\_loss} = \| \tilde{D} \, z \|_2^2 = z^T (\tilde{D}^T \tilde{D}) \, z
$$

##### Proximity Loss
The proximity loss ensures that the optimized path is close to the reference path $z_{\text{ref}}$. 
The Euclidean distance between the optimized path and the reference path is used to define the proximity loss:

$$
\text{proximity\_loss} = \| z - z_{\text{ref}} \|^2 = (z - z_{\text{ref}})^T (z - z_{\text{ref}})
$$

Expanding this term, we obtain:  

$$
\text{proximity\_loss} = z^T z - 2 z_{\text{ref}}^T z + z_{\text{ref}}^T z_{\text{ref}}
$$

##### Objective Function (Combined)
The overall objective is to minimize the weighted sum of the smoothness loss and the proximity loss:
$$
\min_{z} \quad \text{smoothness\_loss} + \lambda \, \text{proximity\_loss}
$$

Substituting the expressions for the smoothness and proximity losses:
$$
\min_{z} \quad z^T (\tilde{D}^T \tilde{D}) \, z + \lambda \left( z^T z - 2 z_{\text{ref}}^T z + z_{\text{ref}}^T z_{\text{ref}} \right)
$$

Reorganize the terms:
$$
\min_{z} \quad z^T (\tilde{D}^T \tilde{D}) \, z + \lambda z^T z - 2 \lambda z_{\text{ref}}^T z + \lambda z_{\text{ref}}^T z_{\text{ref}}
$$

Factor out \( z \) for the quadratic terms:
$$
\min_{z} \quad z^T \left( \tilde{D}^T \tilde{D} + \lambda I \right) z - 2 \lambda z_{\text{ref}}^T z + \lambda z_{\text{ref}}^T z_{\text{ref}}
$$

Where:
- $I \in \mathbb{R}^{2N \times 2N}$ is the identity matrix.  

This is now in the standard QP form, where the objective function is written as:
$$
\min_{z} \quad z^T Q z + q^T z + c
$$

Where:
$$
Q = \tilde{D}^T \tilde{D} + \lambda I
$$
$$
q = -2 \lambda z_{\text{ref}}
$$
$$
c = \lambda z_{\text{ref}}^T z_{\text{ref}}
$$



##### Constraints
The start and end points are fixed to match the original reference path. These are linear equality constraints:  

$$
x[1] = x_0[1], \quad y[1] = y_0[1]
$$

$$
x[N] = x_0[N], \quad y[N] = y_0[N]
$$

These constraints can be written as equality constraints for the variable $z$:

$$
A_{\text{eq}} z = b_{\text{eq}}
$$

Where $A_{\text{eq}} \in \mathbb{R}^{4 \times 2N}$ and $b_{\text{eq}} \in \mathbb{R}^4$.  


To avoid obstacles, we introduce a quadratic inequality constraint for each obstacle.  

Suppose there is an obstacle centered at $\mathbf{z}_{\text{obs}}$ with radius $R$. Then, for any path point $z[i]$, we must ensure:  

$$
\| z[i] - \mathbf{z}_{\text{obs}} \|^2 \geq R^2
$$

This can be rewritten as:  

$$
(z[i] - \mathbf{z}_{\text{obs}})^T (z[i] - \mathbf{z}_{\text{obs}}) \geq R^2
$$

Rearranging, we have:  

$$
z^T (-I) z + 2 \mathbf{z}_{\text{obs}}^T z + (\mathbf{z}_{\text{obs}}^T \mathbf{z}_{\text{obs}} - R^2) \leq 0
$$

For each obstacle, we define:  
$$
Q_i = -I
$$
  
$$
q_i = 2 \mathbf{z}_{\text{obs}}
$$

$$
c_i = \mathbf{z}_{\text{obs}}^T \mathbf{z}_{\text{obs}} - R^2
$$


#### Final QCQP Model
The final QCQP formulation aims to minimize:
$$
\min_z z^T Q z + q^T z + c
$$
Where:
$$
Q = \tilde{D}^T \tilde{D} + \lambda I
$$
$$
q = -2\lambda z_{\text{ref}}
$$
$$
c = \lambda z_{\text{ref}}^T z_{\text{ref}}
$$
Subject to two types of constraints: linear equality constraints:
$$
A_{\text{eq}} z = b_{\text{eq}}
$$
That fix the start and end points, and quadratic inequality constraints:
$$
z^T Q_i z + q_i^T z + c_i \leq 0
$$
For each obstacle i, where:
$$
Q_i = -I
$$
$$
q_i = 2z_{\text{obs}}
$$
$$
c_i = z_{\text{obs}}^T z_{\text{obs}} - R^2
$$

This model optimizes path points while balancing smoothness and proximity to a path, ensuring both endpoint constraints and obstacle avoidance are met.

## 3. Implementation ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **We will be running your code, and you want to make sure that everything works if we run the code blocks in sequence**. Having multiple code blocks separated by text blocks (either as separate cells or blocks of comments) that explain the various parts of your solution will make it much easier for us to understand your project. 

We would also like to see that you make several variants or run different kinds of analysis for your problem (e.g. by changing the input parameters, the constraints or objective, or deriving a dual problem). We expect at least one-two such problem variants as part of your project.

**Remember that if you do not write your description of the project and commeent your code well, we cannot understand what you have done. Even if it is technically brilliant, you will loose points if you do not write well and comment your code well.**
It's fine to call solvers and external packages we have used in the course such as `Gurobi` or `Plots`, but try to minimize the use of other packages. We want to be able to understand what is happening in your code without looking up additional references. 

Now, since we have the implementation of our A* algorithm and smoothed A* algorithm, we can attempt to incorporate it with the original Baldur's Gate map. Using the map, we can convert it to an environment matrix where black space represent obstacles and white space represents feasible area. 

## 4. Results and discussion ##




From our scatter plot we can observe how the A* algorithm works through the Baldur's Gate map. When given the start point, our pathing seems to be close to the quickest path to the destination. While this is a great visualization, we more importantly want to focus on the smoothing optimization we have created to make the game pathing more realistic. In order to do so, we will focus on a few different specific sections of the Baldur's Gate map to get a better idea of how our optimization operates around objects and upon the optimal pathing. 

Given these 3 subsets of the map, we can observe more closely how our smoothing optimizer makes the game movement more realistic. For Section 1, there are long sweeping turns which stand out as opposed to rigid 90 degree turns. In Section 2, our smoothing optimizer very closely avoids an obstacle but still manages to hug it, minimizing proximity loss while also making movement more realistic.

## 5. Conclusion ##

After completing our path smoothing optimization project using Baldur's Gate II maps as our testing ground, we successfully demonstrated how combining A* pathfinding with optimization techniques can transform rigid grid-based paths into smooth, natural-looking trajectories. Our implementation revealed that a smoothness weight (α) of 1.0 and proximity weight (β) of 0.1 provided an effective balance between path smoothing and maintaining efficient routes. Testing on different map segments, particularly in regions (300,350,200,250) and (250,300,150,200), showed that our approach successfully eliminated unrealistic sharp turns while preserving the strategic advantages of optimal pathfinding. The improvement was especially noticeable around obstacles, where our solution created naturally flowing paths instead of the jagged movements typical in traditional grid-based systems. This enhancement directly addresses a common criticism of classic RPGs where character movement often appears artificial and constrained by the underlying grid system.

One key finding from our implementation was the careful balance needed between smoothness and path adherence. When we increased the smoothness weight relative to the proximity weight, the paths exhibited more gradual curves but sometimes deviated too far from optimal routes. This trade-off proved particularly important in game environments, where both movement aesthetics and path efficiency affect player experience. Our segment-based processing approach, using segments of 10 points, successfully managed computational complexity while maintaining path quality across different map regions. Through our experimentation with different parameter combinations, we discovered that local path characteristics significantly influenced the optimal balance between smoothness and proximity constraints. In areas with dense obstacles, stronger adherence to the original path proved beneficial, while open areas benefited from increased smoothing.

Looking toward future developments, an interesting direction would be to implement adaptive parameter tuning based on local path characteristics. Such a system could automatically adjust smoothing parameters based on obstacle density and path complexity, increasing path adherence in crowded areas while allowing for smoother movements in open spaces. This enhancement would be particularly valuable for modern game remakes, where players expect sophisticated character movement while still maintaining strategic depth. By analyzing local environmental conditions, the system could create more context-aware paths that better balance the competing needs of natural movement and efficient navigation. Additionally, this approach could be extended to consider dynamic obstacles and varying terrain types, further enhancing the realism of character movement while preserving the tactical elements that make games like Baldur's Gate engaging. The success of our current implementation provides a strong foundation for these future improvements, demonstrating the potential for optimization techniques to enhance classic game mechanics while meeting modern player expectations.